In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df=pd.read_csv("loan-dataset-clean.csv")

In [3]:
df.head()

,loan_id,principal_loan_amount,request_date,interest_rate,loan_channel,is_approved,is_default,customer_monthly_income,customer_age,customer_gender,customer_credit_score,customer_zip_code
0,70280c1b-4339-4949-ba69-ad2a6cb6b782,8880,2023-03-06,9.0,WEB,True,False,6100,30,M,3,10007
1,8a550559-9749-491b-b6aa-7188bc0a488f,13720,2023-05-10,10.0,MOBILE_APP,True,False,5150,39,F,3,10004
2,5644d27e-e05e-4a40-bef3-64187892338f,2090,2023-02-16,7.5,AGENT,True,False,6030,43,F,3,10003
3,9d8d17ac-8af9-4936-ba5d-8d5ed5e0cfab,6820,2023-02-26,8.5,MOBILE_APP,False,False,2150,54,M,0,10004
4,4586ad7a-2231-419b-b8e3-ba03b929b56e,13770,2023-04-13,10.0,AFFILIATE,False,False,5810,28,F,2,10007


In [5]:
df.isnull().sum()

loan_id                    0
principal_loan_amount      0
request_date               0
interest_rate              0
loan_channel               0
is_approved                0
is_default                 0
customer_monthly_income    0
customer_age               0
customer_gender            0
customer_credit_score      0
customer_zip_code          0
dtype: int64

HYPOTHESIS TESTING


Null hypothesis (from the operations team) : the interest rate has no impact on the loan default rate

Alternative hypothesis : interest rate does have an impact on the loan default rate.

In [8]:
#calculate median of intrest rate ccolumn
median = df['interest_rate'].median()
median

9.0

In [10]:
#splitting dataset into 2 groups based on intrest rates(high and low)
high_interest=df[df['interest_rate']>median]
low_interest=df[df['interest_rate']<median]

In [14]:
high_interest.shape

(5434, 12)

In [15]:
high_interest.size

65208

In [16]:
low_interest.shape

(6829, 12)

In [17]:
low_interest.size

81948

In [18]:
high_default_rate= high_interest['is_default'].mean()
low_default_rate= low_interest['is_default'].mean()

In [19]:
high_default_rate

0.03367684946632315

In [20]:
low_default_rate

0.03704788402401523

In [25]:
t_stat, p_val = stats.ttest_ind(high_interest['is_default'], low_interest['is_default'])

In [26]:
p_val

0.3166597414368534

In [30]:
print('High interest rate default rate: {:.2%}'.format(high_default_rate))
print('Low interest rate default rate: {:.2%}'.format(low_default_rate))

High interest rate default rate: 3.37%
Low interest rate default rate: 3.70%


In [32]:
print('Two-tailed t-test: t-statistic = {:.2f}, p-value = {:.2f}'.format(t_stat, p_val))

Two-tailed t-test: t-statistic = -1.00, p-value = 0.32


In [40]:
df['is_approved'].value_counts()


True     11647
False     3353
Name: is_approved, dtype: int64

In [45]:
import statsmodels.api as sm

In [47]:
income_groups = pd.cut(df['customer_monthly_income'],
                       bins=[0, 2500, 5000, 7500, np.inf])

In [48]:
approval_rate = df.groupby(income_groups)['is_approved'].mean()

In [49]:
X = sm.add_constant(df['customer_monthly_income'])
y = df['is_approved']
model = sm.Logit(y, X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.531339
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            is_approved   No. Observations:                15000
Model:                          Logit   Df Residuals:                    14998
Method:                           MLE   Df Model:                            1
Date:                Sun, 25 Jun 2023   Pseudo R-squ.:               8.514e-06
Time:                        11:11:57   Log-Likelihood:                -7970.1
converged:                       True   LL-Null:                       -7970.2
Covariance Type:            nonrobust   LLR p-value:                    0.7126
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       1.2688      0.067     18.948      0.000       1.138   